In [ ]:
import pandas as pd

In [12]:
dataframe = pd.read_csv("training.1600000.processed.noemoticon.csv",encoding = "ISO-8859-1")

In [18]:
dataframe.columns = ["label","B","C","D","E","tweet"]

In [21]:
dataframe = dataframe[["label","tweet"]]

In [6]:
import re

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
clean_re = re.compile('\W+')
url_re = re.compile("https?://[^\s]+")
hashtag_re = re.compile("#(\w+)")
mention_re = re.compile("@(\w+)")
def preprocessing(text):
    """
    Realiza el preprocesado de un determinado texto:
    1- sustituye las urls por la palabra <url>
    2- sustituye los hashtags por la palabra <hashtag>
    3- sustituye las menciones por la palabra <mencion>
    4- sustituye los numeros por la palabra <numero>
    """
    text_clean = url_re.sub("<url>",text)
    text_clean = hashtag_re.sub("<hashtag>", text_clean)
    text_clean = mention_re.sub("<mencion>", text_clean)
    text_clean =re.sub("\d+", "<numero>", text_clean)
    #text_clean = clean_re.sub(" ",text_clean).lower()
    text_clean = text_clean.lower()
    #text_clean = Stemming(text_clean)
    
    return text_clean

In [26]:
dataframe["preprocessed"] = list(map(preprocessing,dataframe.tweet))

/Users/marcosesteve/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataframe, test_size=0.2)

In [32]:
vectorizer = TfidfVectorizer(ngram_range = (3,5),max_df=0.95,min_df=2,analyzer="char_wb")

In [33]:
bowTrain = vectorizer.fit_transform(train["preprocessed"])
bowTest = vectorizer.transform(test["preprocessed"])

In [37]:
model = LogisticRegression()

In [38]:
model.fit(bowTrain,train.label)

/Users/marcosesteve/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
model.score(bowTest,test.label)

0.8074125

In [43]:
model.predict(vectorizer.transform(["Fuck this economy. I hate aig and their non loan given asses."]))

array([0])